In [6]:
from harvai.data import get_clean_preproc_data

In [54]:
df = get_clean_preproc_data()

In [19]:
df['id'] = df.index
df = df[['article_lowered','id']]

In [20]:
df = df.to_dict(orient='index')

In [21]:
test = []

In [23]:
for key,text in df.items():
    test.append({'id':key,'content':text['article_lowered']})

In [31]:
from haystack.document_stores import InMemoryDocumentStore

document_store = InMemoryDocumentStore()



INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [32]:
articles = df.to_dict()['article_lemmatized']

AttributeError: 'dict' object has no attribute 'to_dict'

In [40]:
{'id':id for id,text in articles.items()}

{'id': 1148}

In [33]:

document_store.write_documents(test)

In [31]:
from haystack.nodes import TfidfRetriever

retriever = TfidfRetriever(document_store)

INFO - haystack.nodes.retriever.sparse -  Found 1149 candidate paragraphs from 1149 docs in DB


In [32]:
candidate_documents = retriever.retrieve(
    query="quelle est la vitesse maximum autorisée sur l'autoroute ? ",
    top_k=10
)

In [33]:
candidate_documents

[<Document: {'content': '  le fait pour tout conducteur d un véhicule autre qu un véhicule à moteur de contrevenir aux dispositions du présent code relatives à la vitesse maximale autorisée est puni de l amende prévue pour les contraventions de la deuxième classe.     section 2 : maîtrise de la vitesse. ', 'content_type': 'text', 'score': None, 'meta': {}, 'embedding': None, 'id': '982'}>,
 <Document: {'content': ' i. - le fait, pour tout conducteur d un véhicule à moteur, de dépasser de moins de 50 km/h la vitesse maximale autorisée fixée par le présent code ou édictée par l autorité investie du pouvoir de police est puni de l amende prévue pour les contraventions de la quatrième classe.     toutefois, lorsque le dépassement est inférieur à 20 km/h et que la vitesse maximale autorisée est supérieure à 50 km/h, l amende encourue est celle prévue pour les contraventions de la troisième classe.      ii. - toute personne coupable de l infraction de dépassement de la vitesse maximale autor

## with embedding retriever

In [11]:
document_store_2 = InMemoryDocumentStore(similarity="dot_product",embedding_dim=768)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0


In [28]:
document_store_2.write_documents(test)

In [19]:
from haystack.nodes import EmbeddingRetriever

In [25]:
retriever = EmbeddingRetriever(
    document_store=document_store_2,
   embedding_model="sentence-transformers/multi-qa-mpnet-base-dot-v1",
   model_format="sentence_transformers"
)

INFO - haystack.modeling.utils -  Using devices: CPU
INFO - haystack.modeling.utils -  Number of GPUs: 0
INFO - haystack.nodes.retriever.dense -  Init retriever using embeddings of model sentence-transformers/multi-qa-mpnet-base-dot-v1
WARNING - haystack.nodes.retriever._embedding_encoder -  You are using a Sentence Transformer with the dot_product function. We recommend using cosine instead. This can be set when initializing the DocumentStore


In [29]:
document_store_2.update_embeddings(retriever)

INFO - haystack.document_stores.memory -  Updating embeddings for 1149 docs ...
Updating Embedding:   0%|          | 0/1149 [00:00<?, ? docs/s]

Batches:   0%|          | 0/36 [00:00<?, ?it/s]

Documents Processed: 10000 docs [08:48, 18.93 docs/s]           


In [30]:
from haystack.utils import print_documents
from haystack.pipelines import DocumentSearchPipeline

p_retrieval = DocumentSearchPipeline(retriever)
res = p_retrieval.run(query="vitesse maximale autorisée sur autoroute ?", params={"Retriever": {"top_k": 10}})
print_documents(res, max_text_len=512)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Query: vitesse maximale autorisée sur autoroute ?

{   'content': 'cas visibilité inférieur mètre vitesse maximal abaissé '
               'kilomètre ensemble réseau routier autoroutier',
    'name': None}

{   'content': 'hors agglomération exception voie lequel vitesse véhiculer '
               'limité kilomètre application article vitesse véhiculer '
               'transport commun limité kilomètre ii toutefois ce vitesse '
               'maximale relevée kilomètre autoroute véhiculer dont poids '
               'total supérieur tonner possédant caractéristique technique '
               'particulier défini arrêté ministre chargé transport autoroute '
               'route chaussée séparer terre plein central véhiculer dont '
               'poids inférieur égal tonner iii exploitation vitesse maximal '
               'abaissé kilomètre autobus autocar passager...',
    'name': None}

{   'content': 'hors agglomération vitesse véhiculer limité kilomètre '
               'autorou

## with elasticsearch

In [29]:
from haystack.document_stores import ElasticsearchDocumentStore

document_store = ElasticsearchDocumentStore(host="localhost", port="9200", username="", password="", index="document")

In [30]:
document_store.write_documents(test)

In [26]:
from haystack.nodes import BM25Retriever

retriever = BM25Retriever(document_store)

In [39]:
candidate_documents = retriever.retrieve(
    query="quelle est la vitesse maximum autorisée en agglomération ?",
    top_k=10
)

In [58]:
list_articles = [int(candidate_documents[id].id) for id in range(0,3)]

In [59]:
list_articles

[982, 979, 1019]

In [55]:
df.columns

Index(['article_base', 'article_number', 'article_content', 'article_lowered',
       'article_wo_numbers', 'article_wo_punctuation', 'article_wo_stopwords',
       'article_tfidf_format', 'article_lemmatized'],
      dtype='object')

In [60]:
' '.join(df.article_lowered[list_articles])
    

'  le fait pour tout conducteur d un véhicule autre qu un véhicule à moteur de contrevenir aux dispositions du présent code relatives à la vitesse maximale autorisée est puni de l amende prévue pour les contraventions de la deuxième classe.     section 2 : maîtrise de la vitesse.   i. - le fait, pour tout conducteur d un véhicule à moteur, de dépasser de moins de 50 km/h la vitesse maximale autorisée fixée par le présent code ou édictée par l autorité investie du pouvoir de police est puni de l amende prévue pour les contraventions de la quatrième classe.     toutefois, lorsque le dépassement est inférieur à 20 km/h et que la vitesse maximale autorisée est supérieure à 50 km/h, l amende encourue est celle prévue pour les contraventions de la troisième classe.      ii. - toute personne coupable de l infraction de dépassement de la vitesse maximale autorisée de 30 km/h ou plus encourt également les peines complémentaires suivantes :     1° la suspension, pour une durée de trois ans au pl

## Testing loading embedding with FAISS Document and DPR

In [1]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever

INFO - haystack.document_stores.base -  Numba not found, replacing njit() with no-op implementation. Enable it with 'pip install numba'.
INFO - haystack.modeling.model.optimization -  apex not found, won't use it. See https://nvidia.github.io/apex/
ERROR - root -  Failed to import 'magic' (from 'python-magic' and 'python-magic-bin' on Windows). FileTypeClassifier will not perform mimetype detection on extensionless files. Please make sure the necessary OS libraries are installed if you need this functionality.


In [2]:
FAISSDocumentStore.load()

AttributeError: type object 'MissingDependency' has no attribute 'load'

In [ ]:

retriever_2 = DensePassageRetriever.load("retriever.pt", document_store=document_store_5)